In [4]:
import mmcv
import json
import pickle
import pandas as pd
from glob import glob
from tqdm import tqdm

In [4]:
tracks = glob('/home/dl18206/Desktop/phd/code/personal/pan-africa-annotation/ape-tracking/tracks/**/*.pkl', recursive=True)
videos = glob('../../acp/videos/core/**/*.mp4', recursive=True)

In [5]:
tracks = glob('/home/dl18206/Desktop/phd/code/personal/pan-africa-annotation/ape-tracking/tracks/**/*.pkl', recursive=True)
videos = glob('../../acp/videos/core/**/*.mp4', recursive=True)

for track in tqdm(tracks):
    
    video_prefix = '../../acp/videos/core'
    filename = track.split('/')[-1].split('.')[0]
    
    with open(track, 'rb') as handle:
        t = pickle.load(handle)
    
    for f in t['annotations']:
        for d in f['detections']:
            d['bbox'] = [x.astype(float) for x in d['bbox']]
            d['detection_score'] = d['detection_score'].astype(float)
    
    with open(f'annotations/{filename}.json', 'w') as handle:
        json.dump(t, handle, ensure_ascii=False, indent=4)

100%|██████████| 5045/5045 [00:55<00:00, 90.36it/s] 


In [60]:
def get_tracklet_df(annotation_path):
    # Generated by single_ape_subset dirs
    annotations = glob(f'{annotation_path}/**/*.json', recursive=True)
    
    ids, tracklet_num, idxs_st = [], [], []

    for a in annotations:
        with open(a, 'rb') as handle:
            data = json.load(handle)
            
        name = data['video']
        
        if not data['annotations']:
            ids.append(name)
            tracklet_num.append(0)
            idxs_st.append([])
            continue
        
        idxs = []

        for frame in data['annotations']:
            if((frame['detections'][0]['ape_id'] == 0) or (frame['detections'][0]['ape_id'] == 1)):
                idxs.append(frame['frame_id'])                
                
        tracklets = 1
        start_idxs = [idxs[0]]

        for i in range(len(idxs)-1):
            if(idxs[i+1] - idxs[i] == 1):
                continue
            else:
                tracklets+=1
                start_idxs.append(idxs[i])
                start_idxs.append(idxs[i+1])
        start_idxs.append(idxs[-1])

        ids.append(name)
        tracklet_num.append(tracklets)
        idxs_st.extend([start_idxs])
    
    df = pd.DataFrame([ids, tracklet_num, idxs_st]).transpose()
    df.columns = ['id', 'tracklet_num', 'indexes']
    return df

In [55]:
def get_count_df(annotation_path):
    # Generated by single_ape_subset dirs
    annotations = glob(f'{annotation_path}/**/*.json', recursive=True)
    
    videos, counts = [], []
    
    for a in annotations:
        with open(a, 'rb') as handle:
            data = json.load(handle)
            
        name = data['video']
        count = count_apes(data)
        
        videos.append(name)
        counts.append(count)

    df = pd.DataFrame([videos, counts]).transpose()
    df.columns = ['videos', 'counts']
    return df

In [56]:
def count_apes(data):
    count = [] 
    for frame in data['annotations']:
        for d in frame['detections']:
            if d['ape_id'] not in count:
                count.append(d['ape_id'])
    return len(count)

In [57]:
count_df = get_count_df(annotation_path='tracked_detections/vfnet_annotations/')

In [61]:
tracklet_df = get_tracklet_df(annotation_path='tracked_detections/vfnet_annotations/')

In [77]:
count_df[count_df.counts==1]

,videos,counts
3,ACP0003bgj,1
4,ACP000dph1,1
8,ACP000ds21,1
9,ACP000drzl,1
10,ACP000b6dx,1
...,...,...
6733,ACP000bco6,1
6735,ACP000bed2,1
6736,ACP000dt51,1
6738,ACP000b3ia,1


In [79]:
import shutil
src_prefix = '/home/dl18206/Desktop/phd/data/panaf/machine_annotations/core/tracked_detections/vfnet_annotations'
dst_prefix = '/home/dl18206/Desktop/phd/data/panaf/machine_annotations/core/tracked_detections/vfnet_single_ape_annotations/'

for i in count_df[count_df.counts==1]['videos'].unique():
    shutil.move(f"{src_prefix}/{i}.json", dst_prefix)

In [76]:
tracklet_df[tracklet_df.tracklet_num==1]

,id,tracklet_num,indexes
3,ACP0003bgj,1,"[211, 360]"
4,ACP000dph1,1,"[270, 360]"
8,ACP000ds21,1,"[1, 360]"
9,ACP000drzl,1,"[1, 360]"
10,ACP000b6dx,1,"[1, 360]"
...,...,...,...
6733,ACP000bco6,1,"[1, 360]"
6735,ACP000bed2,1,"[77, 331]"
6736,ACP000dt51,1,"[1, 360]"
6738,ACP000b3ia,1,"[1, 130]"
